Preparación de los datos para la base que se ingresa al modelo

In [4]:
#Importamos las librerias necesarias
import pandas as pd
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

In [5]:
#cargamos el archivo que contiene los datos, en este caso un archivo de excel
#cargar los datos del archivo de excel en el dataframe
archivo_excel = "/content/Base prospectacion noviembre.xlsx" #aqui va la ruta de donde esta almacenado el archivo
df = pd.read_excel(archivo_excel)

In [6]:
#tamaño inicial del dataframe, primer parametro son la cantidad de registros, segundo la cantidad de columnas
df.shape

(32863, 53)

In [7]:
#codigo para no truncar la salida visual, debido a que se tienen muchas columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [8]:
#revisamos el df cargado, el valor 2 hace referencia a solo mostrar las dos primeros registros
df.head(2)

,TIPO_DE_IDENTIFICACION,IDENTIFICACION,NOMBRE_CLIENTE,DESCRIPCION,FECHA_APERTURA,ESTADO,SALDO_DISPONIBLE,VALOR_CUOTA,ULT_TRANSACCION,FORMA_PAGO,PAGADURIA,PAGADURIA_IDENTIFICACION,CREDIT_VISION,Tipo Riegso,EXPERIENCIA,INCOME_ESTIMATOR,RESULTADO,VALOR_CUOTA_ACTUAL_PRIN,VALOR_SALDO_ACTUAL_PRIN,Solicitudes en Fabrica de Credito,Solicitudes negadas ultimos 3 meses,Saldos de credito,Cupos Rotativos,dias de Mora,Reestructuracion,Riesgo Empresa,Mora Empresa,Delegado,Colaborador,base de julio,Base Hipoteca,Base Septiembre,Gestion Asesor Externo-consumo,Gestion Asesor Externo-retanquo,Gestion Asesor Externo-hipote,GENERO,FECHA_VINCU,ULTIMA_ACTUALIZACION,TIPO_VIVIENDA,ESTRATO,NIVEL_ACADEMICO,ESTADO_CIVIL,PERSONAS_A_CARGO,TOTAL_INGRESOS,TOTAL_ACTIVOS,TOTAL_PASIVOS,PATRIMONIO,NOMBRE_COMPANIA,RELACION_LABORAL,TIPO_CONTRATO,SALARIO,CIUDAD_RESIDENCIA,EDAD
0,CEDULA DE CIUDADANIA,31985279,LAHYD DEL CARMEN DAVID QUINTERO,APORTES SOCIALES,2013-01-24,A,3129477.93,37806.0,2023-10-06,Nomina,BPS CONSULTING S.A.S,901166320.0,981,Bajo,CE,30430507,ACEPTADO,2916000,3514000,0,0,0.0,0,0,0,Alto,0,0,0,Credito Nuevo,0,0,PENDIENTE LLAMADA,0,0,Femenino,2012-02-20,00:00:00,PROPIA,6,Postgrado,Casado,Uno,9000000,740000000,20000000,720000000,BPS CONSULTING S.A.S,Empleados,A tÃ©rmino Indefinido,6000000,BOGOTA,55
1,CEDULA DE CIUDADANIA,46372308,DIANA MILENA SOLANO ROJAS,APORTES SOCIALES,2017-06-14,A,1424025.04,40604.0,2023-10-26,Nomina,SERVICIOS DE AUDITORIA Y CONSULTORIA DE NEGOCI...,800174750.0,798,Medio Bajo,CE,6233439,ACEPTADO,3135000,26385000,0,0,0.0,0,0,0,Alto,0,0,0,0,0,0,0,0,0,Femenino,2005-05-09,00:00:00,PROPIA,4,Universitario,Casado,Uno,3600001,265000000,8000000,257000000,BECKER Y ASSOCIATES S.A.S.,Empleados,A tÃ©rmino Indefinido,3600000,BOGOTA,48


Eliminación de registros no necesarios


In [9]:
# Filtra el DataFrame utilizando query para eliminar los registros diferentes a cero
df = df.query("`Solicitudes en Fabrica de Credito` == 0")

#columna Solicitudes negadas ultimos 3 meses
df = df.query("`Solicitudes negadas ultimos 3 meses` == 0")

# Columna 'ESTADO' es el nombre de tu columna de interés.
df = df.query("`ESTADO` == 'A'")


# Filtra el DataFrame para mantener solo los registros que cumplen todas las condiciones
df = df.loc[
    (df['Cupos Rotativos'] == 0) &
    (df['dias de Mora'] == 0) &
    ((df['Reestructuracion'] == 0) | (df['Reestructuracion'] == 'N')) &
    (df['Mora Empresa'] == 0) &
    (df['Delegado'] == 0) &
    (df['Colaborador'] == 0) &
    (df['Saldos de credito '] < 100000)  # Asegúrate de que el nombre de la columna es exacto
]


In [10]:
#tamaño  del dataframe luego de la eliminacion de registros
df.shape

(26176, 53)

GENERACION DE COLUMNAS A PARTIR DE LAS EXISTENTES

In [11]:
#ANTIGUEDAD EN UNIMOS
# Asegúrate de que las fechas en FECHA_APERTURA estén en formato datetime
df['FECHA_APERTURA'] = pd.to_datetime(df['FECHA_APERTURA'])

# La fecha de referencia contra la cual quieres calcular la antigüedad en meses
# Debes ingresarla en el formato 'YYYY-MM-DD'
fecha_referencia = input("Por favor ingresa la fecha de referencia (YYYY-MM-DD): ")
fecha_referencia = datetime.strptime(fecha_referencia, '%Y-%m-%d')

# Calcula la antigüedad en meses usando una función lambda y relativedelta
df['ANTIGUEDAD_UNIMOS(MESES)'] = df['FECHA_APERTURA'].apply(
    lambda x: (fecha_referencia.year - x.year) * 12 + fecha_referencia.month - x.month - (1 if fecha_referencia.day < x.day else 0)
)




Por favor ingresa la fecha de referencia (YYYY-MM-DD): 2023-11-23


In [12]:
#ANTIGUEDAD LABORAL
# Convertir la columna 'FECHA_VINCU' a datetime, si no lo está ya. Esto fallará si la columna tiene valores de tiempo sin fecha.
df['FECHA_VINCU'] = pd.to_datetime(df['FECHA_VINCU'], errors='raise')

# La fecha de referencia contra la cual quieres calcular la antigüedad
fecha_referencia_str = input("Por favor ingresa la fecha de referencia (YYYY-MM-DD): ")
fecha_referencia = datetime.strptime(fecha_referencia_str, '%Y-%m-%d')

# Calcula la antigüedad en años
df['ANTIGUEDAD_LABORAL_AÑOS'] = df['FECHA_VINCU'].apply(lambda x: (fecha_referencia.year - x.year) - (1 if (fecha_referencia.month, fecha_referencia.day) < (x.month, x.day) else 0))



Por favor ingresa la fecha de referencia (YYYY-MM-DD): 2023-11-27


In [13]:
# Agrega una nueva columna llamada 'plazo de credito' con el valor 60 en todas las filas
df['plazo de credito'] = 60


In [14]:
#CAPACIDAD DE PAGO

def calcular_capacidad_de_pago(row):
    if row['SALARIO'] == 0:
        return 5  # Imputa 5 si el salario es cero para evitar división por cero

    # Calcula la capacidad de pago
    resultado = (row['SALARIO'] - (row['SALARIO'] * 0.36) - row['VALOR_CUOTA_ACTUAL_PRIN']) / row['SALARIO']

    # Imputa valores según los rangos especificados
    if resultado <= 0:
        return 5
    elif 0 < resultado < 0.10:
        return 5
    elif 0.10 <= resultado < 0.30:
        return 4
    elif 0.30 <= resultado < 0.40:
        return 3
    elif 0.40 <= resultado < 0.50:
        return 2
    elif resultado >= 0.50:
        return 1

# Aplica la función y crea la nueva columna 'CAPACIDAD_DE_PAGO'
df['CAPACIDAD_DE_PAGO'] = df.apply(calcular_capacidad_de_pago, axis=1)



In [15]:
#DESCUENTOS DE NOMINA
# Función que calcula el descuento según el salario
def calcular_descuento(salario):
    if salario < 4500000:
        return salario * 0.08
    else:
        return salario * 0.10

# Aplicar la función a la columna 'SALARIO'
df['Descuentos de Nomina'] = df['SALARIO'].apply(calcular_descuento)



In [16]:
#TOTAL GASTOS
# Calcula el nuevo valor en base a la condición dada y lo asigna a una nueva columna
df['TOTAL GASTOS'] = np.where(df['SALARIO'] > 2000000, df['SALARIO'] * 0.30, df['SALARIO'] * 0.40)



In [17]:
#CAPACIDAD DE DESCUENTOS

def calcular_capacidad_descuentos(row):
    # Verificar si el salario es cero para evitar división por cero
    if row['SALARIO'] == 0:
        return 5  # Imputa 5 si el salario es cero

    # Calcula la capacidad de descuentos
    capacidad_descuentos = (row['SALARIO'] * 0.5 - row['Descuentos de Nomina']) / row['SALARIO']

    # Imputación según los rangos especificados
    if capacidad_descuentos <= 0:
        return 5
    elif 0 < capacidad_descuentos < 0.10:
        return 5
    elif 0.10 <= capacidad_descuentos < 0.30:
        return 4
    elif 0.30 <= capacidad_descuentos < 0.40:
        return 3
    elif 0.40 <= capacidad_descuentos < 0.50:
        return 2
    elif capacidad_descuentos >= 0.50:
        return 1

# Aplica la función y crea la nueva columna 'CAPACIDAD_DESCUENTOS'
df['CAPACIDAD_DESCUENTOS'] = df.apply(calcular_capacidad_descuentos, axis=1)



In [18]:
#RAZON GASTOS TOTALES

def calcular_razon_gastos_totales(row):
    # Evita la división por cero asignando un valor por defecto si 'SALARIO' es cero.
    if row['SALARIO'] == 0:
        return 5

    # Calcula la razón de gasto totales
    razon = (row['SALARIO'] + row['TOTAL GASTOS'] + row['Descuentos de Nomina']) / row['SALARIO']

    # Imputación según los rangos especificados
    if razon <= 0 or razon < 0.40:
        return 1
    elif 0.40 <= razon < 0.70:
        return 2
    elif 0.70 <= razon < 0.80:
        return 3
    elif 0.80 <= razon < 0.90:
        return 4
    elif razon >= 0.90:
        return 5

# Aplica la función y crea la nueva columna 'RAZON GASTO TOTALES'
df['RAZON GASTOS TOTALES'] = df.apply(calcular_razon_gastos_totales, axis=1)



In [19]:
# RAZON GASTOS FINANCIEROS

def calcular_razon_gastos_financieros(row):
    # Evita la división por cero asignando un valor por defecto si 'SALARIO' es cero.
    if row['SALARIO'] == 0:
        return 5

    # Calcula la razón de gastos financieros
    razon = row['VALOR_CUOTA_ACTUAL_PRIN'] / row['SALARIO']

    # Imputación según los rangos especificados
    if razon <= 0 or razon < 0.21:
        return 1
    elif 0.21 <= razon < 0.31:
        return 2
    elif 0.31 <= razon < 0.41:
        return 3
    elif 0.41 <= razon < 0.51:
        return 4
    elif razon >= 0.51:
        return 5

# Aplica la función y crea la nueva columna 'RAZON GASTOS FINANCIEROS'
df['RAZON GASTOS FINANCIEROS'] = df.apply(calcular_razon_gastos_financieros, axis=1)



In [20]:
#RAZON ENDEUDAMIENTO TOTAL

def calcular_razon_endeudamiento_total(row):
    # Evita la división por cero asignando un valor por defecto si 'VALOR_SALDO_ACTUAL_PRIN' es cero.
    if row['VALOR_SALDO_ACTUAL_PRIN'] == 0:
        return 5

    # Calcula la razón de endeudamiento total
    razon = row['TOTAL_ACTIVOS'] / row['VALOR_SALDO_ACTUAL_PRIN']

    # Imputación según los rangos especificados
    if razon <= 0 or razon < 0.71:
        return 1
    elif 0.71 <= razon < 0.81:
        return 2
    elif 0.81 <= razon < 0.91:
        return 3
    elif 0.91 <= razon < 1.01:
        return 4
    elif razon >= 1.01:
        return 5

# Aplica la función y crea la nueva columna 'RAZON ENDEUDAMIENTO TOTAL'
df['RAZON ENDEUDAMIENTO TOTAL'] = df.apply(calcular_razon_endeudamiento_total, axis=1)


IMPUTANDO LOS VALORES SEGUN MANUAL

In [21]:
#IMPUTACION ANTIGUEDAD UNIMOS (MESES)
def imputar_antiguedad(meses):
    if meses < 6:
        return 5
    elif 6 <= meses < 12:
        return 4
    elif 12 <= meses < 24:
        return 3
    elif meses == 24:
        return 2
    elif meses >= 25:
        return 1

# Aplicar la función para reemplazar valores en la misma columna
df['ANTIGUEDAD_UNIMOS(MESES)'] = df['ANTIGUEDAD_UNIMOS(MESES)'].apply(imputar_antiguedad)


In [22]:
#ANTIGUEDAD LABORAL
def imputar_antiguedad_laboral(anos):
    if anos <= 1:
        return 5
    elif 1 < anos < 3:
        return 3
    elif 3 <= anos < 5:
        return 2
    elif anos >= 5:
        return 1

# Aplicar la función para reemplazar valores en la misma columna
df['ANTIGUEDAD_LABORAL_AÑOS'] = df['ANTIGUEDAD_LABORAL_AÑOS'].apply(imputar_antiguedad_laboral)


In [23]:
# TIPO VIVIENDA
mapeo_reemplazo = {
 0:4,
"FAMILIAR":5,
"PROPIA":1,
"ARRENDADA":4,

}

# Utiliza el método replace con el diccionario de mapeo
df["TIPO_VIVIENDA"] = df["TIPO_VIVIENDA"].replace(mapeo_reemplazo)

In [24]:
#PUNTAJE SCORING

def imputar_credit_vision(valor):
    if valor <= 0 or valor < 631:
        return 5
    elif 631 <= valor < 701:
        return 4
    elif 701 <= valor < 771:
        return 3
    elif 771 <= valor < 900:
        return 2
    elif valor >= 900:
        return 1

# Aplica la función y crea la nueva columna 'CREDIT_VISION'
df['CREDIT_VISION'] = df['CREDIT_VISION'].apply(imputar_credit_vision)



In [25]:
# imputando la FORMA DE PAGO
mapeo_forma_pago = {
    'Nomina': 3,
    'DÃ©bito Banco': 2,
    'DÃ©bito aut. Externo': 2,
    'DÃ©bito AutomÃ¡tico': 2
}

# Aplicando el mapeo con replace
df['FORMA_PAGO'] = df['FORMA_PAGO'].replace(mapeo_forma_pago)


In [26]:
#VARIABLE RESULTADO
mapeo_resultado = {
    0: 3,
    'ACEPTADO': 1,
    'RECHAZADO': 5,
    'PENDIENTE': 3
}

# Aplicando el mapeo con replace
df['RESULTADO'] = df['RESULTADO'].replace(mapeo_resultado)


In [27]:
#RIESGO EMPRESA

mapeo_riesgo_empresa = {
    0: 3,
    'No aplica': 3,
    'Alto': 5,
    'Medio': 3,
    'Bajo': 1
}

# Aplicando el mapeo con replace
df['Riesgo Empresa'] = df['Riesgo Empresa'].replace(mapeo_riesgo_empresa)


In [28]:
#NIVEL ACADEMICO
mapping = {
    'Bachillerato': 3,
    'Universitario': 4,
    'Técnico': 2,
    'TÃ©cnico':2,
    'Tecnológico': 5,
    'TecnolÃ³gico':5,
    'Primaria': 4,
    'Postgrado': 1,
    'Doctorado': 1,
    'Maestría': 1,
    'MaestrÃ­a':1,
    'Especializacion': 1,
    'Estudiante': 0,
    'Ninguno': 5
}

# Aplica el mapeo a la columna 'NIVEL_ACADEMICO'
df['NIVEL_ACADEMICO'] = df['NIVEL_ACADEMICO'].map(mapping)


In [29]:
# Reemplaza los valores en 'TIPO_CONTRATO' con 'Independiente' cuando 'RELACION_LABORAL' sea 'Independiente'
df.loc[df['RELACION_LABORAL'] == 'Independiente', 'TIPO_CONTRATO'] = 'Independiente'

# Reemplaza los valores en 'TIPO_CONTRATO' con 'Pensionado' cuando 'RELACION_LABORAL' sea 'Pensionado'
df.loc[df['RELACION_LABORAL'] == 'Pensionado', 'TIPO_CONTRATO'] = 'Pensionado'


In [30]:
#TIPO DE CONTRATO
# Define un diccionario para mapear los tipos de contrato a los valores deseados
mapping = {
    'A término Indefinido': 2,
    'A tÃ©rmino Indefinido':2,
    'Otra': 5,
    'A término Fijo': 3,
    'A tÃ©rmino Fijo':3,
    'Independiente': 3,
    'Pensionado': 1,
    'Honorarios': 5,
    'Otra-': 5,
    '0': 3,
    "INDEFINIDO":2,
"FIJO":3,
"LABOR CONTRATADA":5,
"PRESTACION DE SERVICIOS":5,
"INDEPENDIENTE":3,
"PENSIONADO":1,
"TEMPORAL":3,
"CONTRATISTA":4
}

# Aplica el mapeo a la columna 'TIPO_CONTRATO'
df['TIPO_CONTRATO'] = df['TIPO_CONTRATO'].map(mapping)



In [31]:
#codigo para reemplazar segun la edad y de acuerdo al rango de edad

def imputar_edad(edad):
    if 18 <= edad <= 24:
        return 5
    elif 25 <= edad <= 36:
        return 4
    elif 37 <= edad <= 48:
        return 3
    elif 49 <= edad <= 60:
        return 2
    elif edad >= 61:
        return 1
    else:
        # Retorna el valor original si no cae en ningún rango especificado
        return edad

# Aplicando la función imputar_edad a la columna EDAD
df['EDAD'] = df['EDAD'].apply(imputar_edad)


In [32]:
#PERSONAS A CARGO
#Define un diccionario para mapear las descripciones de personas a cargo a los valores deseados
mapping = {
    'Cero': 1,
    'Uno': 2,
    'Dos': 2,
    'Tres': 3,
    'Cuatro': 5,
    '0': 3
}

# Para manejar "cuatro o más", necesitamos aplicar una función personalizada, ya que este caso no es directo
def map_personas_a_cargo(value):
    if value in mapping:
        return mapping[value]
    else:
        # Asume que cualquier valor no mapeado directamente es "cuatro o más"
        return 5

# Aplica la función a la columna 'PERSONAS_A_CARGO'
df['PERSONAS_A_CARGO'] = df['PERSONAS_A_CARGO'].apply(map_personas_a_cargo)


In [33]:
#Imputamos los valores de la variable plazo solicitado meses

def impute_plazo(val):
    if 0 <= val <= 24:
        return 1
    elif 25 <= val <= 36:
        return 2
    elif 37 <= val <= 48:
        return 3
    elif 49 <= val <= 60:
        return 4
    elif val >= 61:
        return 5

# Aplicando la función de imputación y sobrescribiendo la columna PLAZO_SOLICITADO_MESES
df['plazo de credito'] = df['plazo de credito'].apply(impute_plazo)

Eliminación de columnas no necesarias

1.   Columnas agregadas de manera informativa
2.   Columnas despues de usadas para calculos




In [34]:
df.columns

Index(['TIPO_DE_IDENTIFICACION', 'IDENTIFICACION', 'NOMBRE_CLIENTE',
       'DESCRIPCION', 'FECHA_APERTURA', 'ESTADO', 'SALDO_DISPONIBLE',
       'VALOR_CUOTA', 'ULT_TRANSACCION', 'FORMA_PAGO', 'PAGADURIA',
       'PAGADURIA_IDENTIFICACION', 'CREDIT_VISION', 'Tipo Riegso',
       'EXPERIENCIA', 'INCOME_ESTIMATOR', 'RESULTADO',
       'VALOR_CUOTA_ACTUAL_PRIN', 'VALOR_SALDO_ACTUAL_PRIN',
       'Solicitudes en Fabrica de Credito',
       'Solicitudes negadas ultimos 3 meses', 'Saldos de credito ',
       'Cupos Rotativos', 'dias de Mora', 'Reestructuracion', 'Riesgo Empresa',
       'Mora Empresa', 'Delegado', 'Colaborador', 'base de julio',
       'Base Hipoteca', 'Base Septiembre', 'Gestion Asesor Externo-consumo',
       'Gestion Asesor Externo-retanquo', 'Gestion Asesor Externo-hipote',
       'GENERO', 'FECHA_VINCU', 'ULTIMA_ACTUALIZACION', 'TIPO_VIVIENDA',
       'ESTRATO', 'NIVEL_ACADEMICO', 'ESTADO_CIVIL', 'PERSONAS_A_CARGO',
       'TOTAL_INGRESOS', 'TOTAL_ACTIVOS', 'TOTAL_PASI

In [36]:
#Codigo para quitar las columnas que no van a hacer parte del archivo con las columnas del modelo
#se crea una lista con los nombres de las columnas a eliminar

columnas_a_eliminar = ['TIPO_DE_IDENTIFICACION', 'NOMBRE_CLIENTE',
       'DESCRIPCION', 'FECHA_APERTURA', 'ESTADO', 'SALDO_DISPONIBLE',
       'VALOR_CUOTA', 'ULT_TRANSACCION', 'PAGADURIA',
       'PAGADURIA_IDENTIFICACION',
       'EXPERIENCIA', 'INCOME_ESTIMATOR',
       'VALOR_CUOTA_ACTUAL_PRIN', 'VALOR_SALDO_ACTUAL_PRIN',
       'Solicitudes en Fabrica de Credito',
       'Solicitudes negadas ultimos 3 meses', 'Saldos de credito ',
       'Cupos Rotativos', 'dias de Mora', 'Reestructuracion',
       'Mora Empresa', 'Delegado', 'Colaborador', 'base de julio',
       'Base Hipoteca', 'Base Septiembre', 'Gestion Asesor Externo-consumo',
       'Gestion Asesor Externo-retanquo', 'Gestion Asesor Externo-hipote',
       'GENERO', 'FECHA_VINCU', 'ULTIMA_ACTUALIZACION', 'ESTRATO', 'ESTADO_CIVIL',
       'TOTAL_INGRESOS', 'TOTAL_ACTIVOS', 'TOTAL_PASIVOS', 'PATRIMONIO',
       'NOMBRE_COMPANIA', 'RELACION_LABORAL', 'SALARIO',
       'CIUDAD_RESIDENCIA','Descuentos de Nomina','TOTAL GASTOS','Tipo Riegso']


# Eliminando las columnas especificadas directamente del DataFrame original
df.drop(columnas_a_eliminar, axis=1, inplace=True)

In [37]:
df.columns = [col.upper() for col in df.columns]

In [38]:
#CAMBIO DE NOMBRE A LAS VARIABLES
# Crear un diccionario con los nombres antiguos y los nuevos
nuevos_nombres = {
    "IDENTIFICACION": "ID",
    "EDAD": "EDAD",
    "NIVEL_ACADEMICO": "NIVEL EDUCATIVO",
    "TIPO_VIVIENDA": "TIPO VIVIENDA",
    "TIPO_CONTRATO": "TIPO CONTRATO",
    "ANTIGUEDAD_LABORAL_AÑOS": "ANTIGUEDAD",
    "PERSONAS_A_CARGO": "PERSONAS A CARGO",
    "CREDIT_VISION": "CLEAR_SCORE",
    "RESULTADO": "RESULTADO_PRESICION",
    "FORMA_PAGO": "TIPO_CREDITO",
    "PLAZO DE CREDITO": "PLAZO_SOLICITADO_MESES",
    "ANTIGUEDAD_UNIMOS(MESES)": "ANTIGUEDAD EN UNIMOS",
    "CAPACIDAD_DE_PAGO": "CAPACIDAD DE PAGO",
    "CAPACIDAD_DESCUENTOS": "CAPACIDAD DE DESCUENTO",
    "RAZON GASTOS TOTALES": "RAZON GASTOS TOTALES",
    "RAZON GASTOS FINANCIEROS": "RAZON GASTOS FINANCIEROS",
    "RAZON ENDEUDAMIENTO TOTAL": "RAZON ENDEUDAMIENTO TOTAL",
    "RIESGO EMPRESA": "RIESGO EMPRESA"
}

# Renombrar las columnas en el DataFrame
df.rename(columns=nuevos_nombres, inplace=True)




In [39]:
# Lista de las columnas en el orden deseado
orden_columnas = [
    "ID",
    "EDAD",
    "NIVEL EDUCATIVO",
    "TIPO VIVIENDA",
    "TIPO CONTRATO",
    "ANTIGUEDAD",
    "PERSONAS A CARGO",
    "CLEAR_SCORE",
    "RESULTADO_PRESICION",
    "TIPO_CREDITO",
    "PLAZO_SOLICITADO_MESES",
    "ANTIGUEDAD EN UNIMOS",
    "CAPACIDAD DE PAGO",
    "CAPACIDAD DE DESCUENTO",
    "RAZON GASTOS TOTALES",
    "RAZON GASTOS FINANCIEROS",
    "RAZON ENDEUDAMIENTO TOTAL",
    "RIESGO EMPRESA"
]

# Reordenar las columnas en el DataFrame
df = df[orden_columnas]


In [40]:
#Cargar los datos del dataframe actual a un archivo de excel para luego descargar el file

nombre_del_archivo = 'BD_nuevos_para_el_modelo.xlsx'

#Utiliza la función to_excel() para exportar el DataFrame a Excel
df.to_excel(nombre_del_archivo, index=False)  # El argumento index=False evita incluir el índice en el archivo